In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import json
import pandas as pd
import os
import sys
import cv2
import numpy as np
sys.path.append('..')
import matplotlib.pyplot as plt
from utils import plotly_utils
from fidelity_check import fidelity_detect, check_utils
import copy

MAX_BR = 5

def compare_screenshot(live_img, archive_img):
    if not os.path.exists(live_img) or not os.path.exists(archive_img):
        return -1
    img1 = cv2.imread(live_img)
    img2 = cv2.imread(archive_img)
    height = min(img1.shape[0], img2.shape[0])
    width = min([img1.shape[1], img2.shape[1]])
    img1 = img1[:height,:width,:]
    img2 = img2[:height,:width,:]
    diff = img1 - img2
    total = img1.shape[0]*img1.shape[1]*img1.shape[2]
    same = np.count_nonzero(diff == 0)
    return same / total, diff

### Check the assumption of archive's writes are always a subset of live

### Fidelity check

In [12]:
dirr = '../revert_rewrite/writes_carta_check_test/www.mnglassart.org_1063_3_a173c01bea4eb8aa94af54334f5b89af13f8d20b3d460a6cfcc439ddd5786766'
# dirr = '../revert_rewrite/test/load_override/writes/test'
results = json.load(open(os.path.join(dirr, 'results.json'), 'r'))
stage, fixedIdx = None, None
for stage, result in results.items():
    if result['fixedIdx'] == -1:
        continue
    fixedIdx = result['fixedIdx']
    break
# stage = 'interaction_2'
# result = results[stage]
# fixedIdx = result['fixedIdx']
print("stage", stage)
print("fixedIdx", fixedIdx)
left = f'{stage}_initial'
right = f'{stage}_exception_{fixedIdx}'
print("Elements length", len(json.load(open(os.path.join(dirr, f'{left}_elements.json')))), len(json.load(open(os.path.join(dirr, f'{right}_elements.json')))))
print("Writes length", len(json.load(open(os.path.join(dirr, f'{left}_writes.json')))['rawWrites']), 
                        len(json.load(open(os.path.join(dirr, f'{right}_writes.json')))['rawWrites']))
issue, (left_u, right_u) = fidelity_detect.fidelity_issue(dirr, left, right)
print(issue)
json_left_u = copy.deepcopy(left_u)
for i in range(len(json_left_u)):
    json_left_u[i] = json_left_u[i][:MAX_BR] + ['...'] if len(json_left_u[i]) > MAX_BR else json_left_u[i]
json_right_u = copy.deepcopy(right_u)
for i in range(len(json_right_u)):
    json_right_u[i] = json_right_u[i][:MAX_BR] + ['...'] if len(json_right_u[i]) > MAX_BR else json_right_u[i]
print(json.dumps(json_left_u, indent=2))
print(json.dumps(json_right_u, indent=2))

stage load
fixedIdx 0_3
Elements length 780 469
Writes length 669 670
False
[]
[]


### Screenshot similarity (on results)

In [ ]:
# dirr = '../revert_rewrite/writes_carta_check_test/www.mnglassart.org_1063_3_a173c01bea4eb8aa94af54334f5b89af13f8d20b3d460a6cfcc439ddd5786766'
dirr = '../datacollect/ground-truth/determinism/www.dreamhost.com_d45c43ce8b'

results = json.load(open(os.path.join(dirr, 'results.json'), 'r'))
stage, fixedIdx = None, None
for stage, result in results.items():
    if result['fixedIdx'] == -1:
        continue
    fixedIdx = result['fixedIdx']
    break

left = f'{dirr}/{stage}_initial.png'
right = f'{dirr}/{stage}_exception_{fixedIdx}.png'

print(left)
print(right)

simi, diff = compare_screenshot(left, right)
print(simi)
cv2.imwrite(os.path.join(dirr, 'diff.png'), diff)

### Pure screenshot cmp

In [29]:
dirr = '../../fidelity-files/writes/gt_tranco/yapolitic.ru_18031db01e'
left = 'live.png'
right = 'proxy.png'

simi, diff = compare_screenshot(os.path.join(dirr, left), os.path.join(dirr, right))
print(simi)
cv2.imwrite(os.path.join(dirr, 'diff.png'), diff)

0.9999387292892233


True

### Pure layout tree cmp

In [28]:
dirr = '../../fidelity-files/writes/gt_tranco/yapolitic.ru_18031db01e'
left = 'live_elements.json'
right = 'proxy_elements.json'

left_element = json.load(open(f"{dirr}/{left}"))
right_element = json.load(open(f"{dirr}/{right}"))
left_element, right_element = fidelity_detect.dedeup_elements(left_element), fidelity_detect.dedeup_elements(right_element)
left_unique, right_unique = check_utils.diff(left_element, right_element, returnHTML=False)
print(json.dumps((left_unique, right_unique), indent=2))

[
  [],
  []
]


### Diff writes

In [30]:
dirr = '../revert_rewrite/writes/www.nytimes.com_1073_3_aebb2a70fcc39ea0202fcedcc6e8c81852149174e479d139da024d11887cf02b'
# dirr = '../revert_rewrite/test/load_override/writes/grunt.ca'
stage = 'load'
idx = '0_2'
attribute = 'rawWrites'
left_writes = json.load(open(os.path.join(dirr, f'{stage}_initial_writes.json')))
right_writes = json.load(open(os.path.join(dirr, f'{stage}_exception_{idx}_writes.json')))
print("Length of writes", len(left_writes[attribute]), len(right_writes[attribute]))
left_unique, right_unique = check_utils.diff_writes(left_writes, right_writes, attribute=attribute)
# len(left_unique), len(right_unique)
list(left_unique.keys()), list(right_unique.keys())

Length of writes 42 43


([('head', 'script', 'appendChild'),
  ('head', 'style', 'insertBefore'),
  ('div', 'div', 'removeChild')],
 [('head', 'link', 'appendChild'),
  ('div', 'div', 'appendChild'),
  ('n.length;s++)o=n', 'iframe', '#text', 'insertBefore')])

In [ ]:
fixed = json.load(open('../revert_rewrite/fixed_count.json'))
attribute = 'rawWrites'
for dirr, stage in fixed.items():
    print(dirr, stage)
    all_dirr = f'../revert_rewrite/writes/{dirr}'
    result = json.load(open(os.path.join(all_dirr, 'results.json'), 'r'))[stage]
    idx = result['fixedIdx']
    left_writes = json.load(open(os.path.join(all_dirr, f'{stage}_initial_writes.json')))
    right_writes = json.load(open(os.path.join(all_dirr, f'{stage}_exception_{idx}_writes.json')))
    left_unique, right_unique = check_utils.diff_writes(left_writes, right_writes, attribute=attribute)
    print(len(left_unique), len(right_unique))

### fidelity check on a set of loads

In [14]:
def valid_dir(dirr):
    valid = os.path.exists(f'{dirr}/live_elements.json') and os.path.exists(f'{dirr}/archive_elements.json')
    return valid

base = '../record_replay/writes'
data = json.load(open(f'{base}/fidelity_check_metadata.json', 'r'))
results = []
for url, info in list(data.items()):
    dirr = info['directory']
    if not valid_dir(f"{base}/{dirr}"):
        continue
    issue, _ =  fidelity_detect.fidelity_issue(f"{base}/{dirr}")
    issue_screenshot, simi = fidelity_detect.fidelity_issue_screenshot(f"{base}/{dirr}")
    results.append({
        'directory': dirr,
        'issue': issue,
        'issue_screenshot': issue_screenshot,
        'screenshot_similarity': simi
    })
json.dump(results, open('fidelity_check.json', 'w+'), indent=2)

### fidelity check (single)

In [39]:
base = '/vault-swift/jingyz/fidelity-files/writes/ground_truth/'
dirr = 'dceoapps.ildceo.net_91c212044d'
issue, (left_u, right_u) = fidelity_detect.fidelity_issue(os.path.join(base, dirr), 'live', 'archive', meaningful=True)
print(issue)
print(json.dumps(left_u, indent=2))
print(json.dumps(right_u, indent=2))

False
[]
[]


### Compare my impact score vs screenshot

In [ ]:
impacts = json.load(open('fidelity_impact.json', 'r'))
imports_gt = json.load(open('fidelity_impact_gt.json', 'r'))
d = {'ground_truth': [r['impact'] for r in imports_gt], 'detection': [r['impact'] for r in impacts]}
df = pd.DataFrame({k: pd.Series(v) for k, v in d.items()})
plotly_utils.plot_CDF_log(df, xtitle='Impact score (No. pixels)')
    

### Heatmap

In [ ]:
array = np.load('fidelity_impact_heatmap_gt.npy')
plotly_utils.plot_heatmap(array, 'Fidelity impact heatmap (Ground truth)')

array = np.load('fidelity_impact_heatmap.npy')
plotly_utils.plot_heatmap(array, 'Fidelity impact heatmap (Detection)')